# Atmospheric refraction examples 
### PyRugged API - tutorial 4
---

Module imports

In [1]:
import orekit
import numpy as np

import os
import sys


from org.hipparchus.geometry.euclidean.threed import Rotation
from org.hipparchus.geometry.euclidean.threed import Vector3D
from org.hipparchus.stat.descriptive import DescriptiveStatistics

from org.orekit.frames import Frame
from org.orekit.frames import FramesFactory
from org.orekit.frames import Transform

from org.orekit.time import AbsoluteDate
from org.orekit.time import TimeScale
from org.orekit.time import TimeScalesFactory
from org.orekit.utils import AngularDerivativesFilter
from org.orekit.utils import CartesianDerivativesFilter
from org.orekit.utils import IERSConventions
from org.orekit.utils import PVCoordinates
from org.orekit.utils import TimeStampedAngularCoordinates
from org.orekit.utils import TimeStampedPVCoordinates

from pyrugged.configuration.init_orekit import init_orekit
from pyrugged.intersection.algorithm_id import AlgorithmId
from pyrugged.intersection.intersection_algorithm import create_intersection_algorithm
from pyrugged.bodies.body_rotating_frame_id import BodyRotatingFrameId
from pyrugged.bodies.ellipsoid_id import EllipsoidId
from pyrugged.model.inertial_frame_id import InertialFrameId
from pyrugged.model.pyrugged_builder import PyRuggedBuilder
from pyrugged.line_sensor.line_sensor import LineSensor
from pyrugged.line_sensor.linear_line_datation import LinearLineDatation
from pyrugged.los.fixed_rotation import FixedRotation
from pyrugged.los.los_builder import LOSBuilder
from pyrugged.refraction.atmospheric_refraction import AtmosphericRefraction
from pyrugged.refraction.multi_layer_model import MultiLayerModel
from pyrugged.bodies.geodetic_point import GeodeticPoint
from pyrugged.location.optical import OpticalLocation, CorrectionsParams
from pyrugged.utils.math_utils import compute_distance_in_meter_from_points

# add helpers modules from tests
import os
import sys
module_path = os.path.join(os.getcwd(), "./../tests/")
if module_path not in sys.path:
    sys.path.append(module_path)

import helpers

### Initialization 

initialize JVM or GraalVM subtrat VM, and initialize Orekit, using internal orekit-data folder.

In [2]:
init_orekit()

## 1 - Sensor's definition

### Line of sight
The raw viewing direction of pixel i with respect to the instrument is defined by the vector:

In [3]:
raw_dirs = []
for i in range(2000):
    raw_dirs.append(
        Vector3D(0.0, i * float(np.radians(20.0)) / 2000.0, 1.0)
    )

The instrument is oriented 10° off nadir around the X-axis. 
We need to rotate the viewing direction to obtain the line of sight in the satellite frame.

In [4]:
los_builder = LOSBuilder(raw_dirs)
los_builder.add_ti_los_transform(
    FixedRotation(
        "10-degrees-rotation",
        Vector3D.PLUS_I,
        float(np.radians(10.))
    )
)

line_of_sight = los_builder.build()

### Datation model
We use Orekit for handling time and dates, and PyRugged for defining the datation model:

In [5]:
gps = TimeScalesFactory.getGPS()
abs_date = AbsoluteDate("2009-12-11T16:59:30.0", gps)
line_datation = LinearLineDatation(
    abs_date, 1.0, 20
)

### Line sensor
With the LOS and the datation now defined, we can initialize a line sensor object in PyRugged:

In [6]:
line_sensor = LineSensor(
    "my-sensor",
    line_datation,
    Vector3D.ZERO,
    line_of_sight,
)

## 2 - Satellite position, velocity and attitude

### Reference frames
In our application, we simply need to know the name of the frames we are working with. Positions and
velocities are given in the ITRF terrestrial frame, while the quaternions are given in EME2000
inertial frame.

In [7]:
eme_2000 = FramesFactory.getEME2000()

# We don't want to compute tiny tidal effects at millimeter level
simple_eop = True

itrf = FramesFactory.getITRF(
    IERSConventions.IERS_2010, simple_eop
)

### Satellite attitude

In [8]:
satellite_q_list = []

helpers.add_satellite_q(gps, satellite_q_list, "2009-12-11T16:58:42.592937", -0.340236, 0.333952, -0.844012, -0.245684)
helpers.add_satellite_q(gps, satellite_q_list, "2009-12-11T16:59:06.592937", -0.354773, 0.329336, -0.837871, -0.252281)
helpers.add_satellite_q(gps, satellite_q_list, "2009-12-11T16:59:30.592937", -0.369237, 0.324612, -0.831445, -0.258824)
helpers.add_satellite_q(gps, satellite_q_list, "2009-12-11T16:59:54.592937", -0.3836, 0.319792, -0.824743, -0.265299)
helpers.add_satellite_q(gps, satellite_q_list, "2009-12-11T17:00:18.592937", -0.397834, 0.314883, -0.817777, -0.271695)
helpers.add_satellite_q(gps, satellite_q_list, "2009-12-11T17:00:42.592937", -0.411912, 0.309895, -0.810561, -0.278001)
helpers.add_satellite_q(gps, satellite_q_list, "2009-12-11T17:01:06.592937", -0.42581, 0.304838, -0.803111, -0.284206)
helpers.add_satellite_q(gps, satellite_q_list, "2009-12-11T17:01:30.592937", -0.439505, 0.299722, -0.795442, -0.290301)
helpers.add_satellite_q(gps, satellite_q_list, "2009-12-11T17:01:54.592937", -0.452976, 0.294556, -0.787571, -0.296279)
helpers.add_satellite_q(gps, satellite_q_list, "2009-12-11T17:02:18.592937", -0.466207, 0.28935, -0.779516, -0.302131)


### Positions and velocities

In [9]:
satellite_pv_list = []

helpers.add_satellite_pv(gps, eme_2000, itrf, satellite_pv_list, "2009-12-11T16:58:42.592937", -726361.466, -5411878.485, 4637549.599, -2463.635, -4447.634, -5576.736)
helpers.add_satellite_pv(gps, eme_2000, itrf, satellite_pv_list, "2009-12-11T16:59:04.192937", -779538.267, -5506500.533, 4515934.894, -2459.848, -4312.676, -5683.906)
helpers.add_satellite_pv(gps, eme_2000, itrf, satellite_pv_list, "2009-12-11T16:59:25.792937", -832615.368, -5598184.195, 4392036.13, -2454.395, -4175.564, -5788.201)
helpers.add_satellite_pv(gps, eme_2000, itrf, satellite_pv_list, "2009-12-11T16:59:47.392937", -885556.748, -5686883.696, 4265915.971, -2447.273, -4036.368, -5889.568)
helpers.add_satellite_pv(gps, eme_2000, itrf, satellite_pv_list, "2009-12-11T17:00:08.992937", -938326.32, -5772554.875, 4137638.207, -2438.478, -3895.166, -5987.957)
helpers.add_satellite_pv(gps, eme_2000, itrf, satellite_pv_list, "2009-12-11T17:00:30.592937", -990887.942, -5855155.21, 4007267.717, -2428.011, -3752.034, -6083.317)
helpers.add_satellite_pv(gps, eme_2000, itrf, satellite_pv_list, "2009-12-11T17:00:52.192937", -1043205.448, -5934643.836, 3874870.441, -2415.868, -3607.05, -6175.6)
helpers.add_satellite_pv(gps, eme_2000, itrf, satellite_pv_list, "2009-12-11T17:01:13.792937", -1095242.669, -6010981.571, 3740513.34, -2402.051, -3460.291, -6264.76)
helpers.add_satellite_pv(gps, eme_2000, itrf, satellite_pv_list, "2009-12-11T17:01:35.392937", -1146963.457, -6084130.93, 3604264.372, -2386.561, -3311.835, -6350.751)
helpers.add_satellite_pv(gps, eme_2000, itrf, satellite_pv_list, "2009-12-11T17:01:56.992937", -1198331.706, -6154056.146, 3466192.446, -2369.401, -3161.764, -6433.531)
helpers.add_satellite_pv(gps, eme_2000, itrf, satellite_pv_list, "2009-12-11T17:02:18.592937", -1249311.381, -6220723.191, 3326367.397, -2350.574, -3010.159, -6513.056)

## 3 - PyRugged initialization

In [10]:
builder = PyRuggedBuilder()

builder.set_ellipsoid(
    new_ellipsoid = None,
    ellipsoid_id = EllipsoidId.WGS84,
    body_rotating_frame_id = BodyRotatingFrameId.ITRF,
)

builder.set_time_span(
    abs_date,
    abs_date.shiftedBy(60.0),
    0.01,
    5 / line_sensor.get_rate(0)
)

builder.set_trajectory(
    satellite_pv_list,
    4, CartesianDerivativesFilter.USE_P,
    satellite_q_list, 
    4, AngularDerivativesFilter.USE_R,
    inertial_frame_id=InertialFrameId.EME2000,
)

builder.add_line_sensor(line_sensor)

rugged = builder.build()

## 4 -  Intersection algorithm with atmospheric refraction model

In [11]:
# Defines atmospheric refraction model (with the default multi layers model)
atmospheric_refraction = MultiLayerModel(rugged.ellipsoid)

# One can use its own atmospheric model that must extend the abstract refraction.AtmosphericRefraction class.

# Set the optional grid steps, for the inverse location computation. Useless for direct location computation.
# This setting is optional as default values exist. But can be useful for instance to improve results by given smaller steps 
# or to speed up the computation bigger steps. 
pixel_step = 100
line_step = 100
atmospheric_refraction.set_grid_steps(pixel_step, line_step)

correction_params = CorrectionsParams(False, False, atmospheric_refraction)
intersection_algorithm = create_intersection_algorithm(AlgorithmId.IGNORE_DEM_USE_ELLIPSOID)

location = OpticalLocation(rugged, intersection_algorithm, correction_params)


correction_params = CorrectionsParams(False, False, None)


location_without_refraction = OpticalLocation(rugged, intersection_algorithm, correction_params)

## 4 - Direct location WITH and WITHOUT atmospheric correction

In [12]:
def compute_distance_in_meter(earth_radius, gp_1, gp_2):
    """Compute distance in meters between two GeodeticPoint objects."""

    p_1 = Vector3D(gp_1.longitude, gp_1.latitude)
    p_2 = Vector3D(gp_2.longitude, gp_1.latitude)

    return earth_radius * Vector3D.angle(p_1, p_2)


chosen_line = 200

# With atmosphere
gp_with_atmospheric_refraction_correction = location.direct_location_of_sensor_line(chosen_line)

# Without atmosphere
gp_without_atmospheric_refraction_correction = location_without_refraction.direct_location_of_sensor_line(chosen_line)


earth_radius = rugged.ellipsoid.equatorial_radius
stat_distance = []

for index, _ in enumerate(gp_with_atmospheric_refraction_correction):
    current_radius = earth_radius + (
        gp_with_atmospheric_refraction_correction[index][2] + gp_without_atmospheric_refraction_correction[index][2]
    ) / 2.
    
    distance = compute_distance_in_meter_from_points(
        current_radius,
        gp_with_atmospheric_refraction_correction[index],
        gp_without_atmospheric_refraction_correction[index],
    )

    stat_distance.append(distance)

print(f'{"Distance must be > 0 and < 2m:"}')
print(f'{"Max = "}{np.max(stat_distance)}')
print(f'{"Min = "}{np.min(stat_distance)}')
print(f'{"Median = "}{np.median(stat_distance)}')
print(f'{"Mean = "}{np.mean(stat_distance)}')
print(f'{"Std deviation = "}{np.std(stat_distance)}')

Distance must be > 0 and < 2m:
Max = 0.5669886864451367
Min = 0.00021678107073320137
Median = 0.25510454060494187
Mean = 0.25899266436729373
Std deviation = 0.15350403049652708


## 6 - Inverse loc WITH atmospheric correction

In [13]:
epsilon_pixel = 1.0e-3
epsilon_line = 1.1e-2

min_line = int(np.floor(line_sensor.get_line(rugged.min_date)))
max_line = int(np.ceil(line_sensor.get_line(rugged.max_date)))

for index, _ in enumerate(gp_with_atmospheric_refraction_correction):
    
    # to check if we go back to the initial point when taking the geodetic point with atmospheric correction
    gp_with = gp_with_atmospheric_refraction_correction[index]
    line, pixel = location.inverse_location(min_line, max_line, gp_with[1], gp_with[0], gp_with[2])

    if line is not None and pixel is not None:
        # Compare the computed pixel vs expected
        if np.abs(index - pixel) > epsilon_pixel:
            print(f'{"Problem with pixel "}{index}{" . Delta pixel = "}{index - pixel}')

        # Compare the computed line vs the expected
        if np.abs(chosen_line - line) > epsilon_line:
            print(f'{"Problem with line, for pixel "}{index}{" . Delta line = "}{chosen_line - line}')

    else:
        print(f'{"Inverse location failed for pixel "}{index}{" with atmospheric refraction correction for geodetic point computed with"}')
    
print(
    f'{"Inverse location gave same pixel and line as direct location input for :"}'
    f'{" Epsilon pixel = "}{epsilon_pixel}'
    f'{" Epsilon line = "}{epsilon_line}'
)

Inverse location gave same pixel and line as direct location input for : Epsilon pixel = 0.001 Epsilon line = 0.011
